Imports and setup

In [1]:
%load_ext autoreload
%autoreload 2

from typing import Any
from enum import Enum
import csv

from thefuzz import process, fuzz

import utils

See what entries in Notion can updated

In [2]:
celeb_heights_filename = r'.\data\celeb-heights-2025-03-13.csv'


class CelebHeightsEntry:
    def __init__(self, lst):
        self.name: str = lst[0]
        self.height: int = int(lst[1]) if lst[1] else None
        self.image_url = lst[2]
        self.url = lst[3]

class Outdated(Enum):
    HEIGHT = 'height'
    IMAGE = 'image'

class OutdatedWrapper:
    def __init__(self, outdated, current_val, should_be_val):
        self.outdated: Outdated = outdated
        self.current_val: Any = current_val
        self.should_be_val: Any = should_be_val

class UpdateThis:
    def __init__(self, entry, celeb_heights_entry, percent_match):
        self.notion_entry: utils.Entry = entry
        self.celeb_heights_entry: CelebHeightsEntry = celeb_heights_entry
        self.percent_match = percent_match
        self.updates_needed: list[OutdatedWrapper] = []

    def add_update_needed(self, update):
        self.updates_needed.append(update)

    def needs_update(self) -> bool:
        return len(self.updates_needed) > 0


# import all celeb heights entries
celeb_heights_entries_map: dict[str, CelebHeightsEntry] = dict()
with open(celeb_heights_filename, mode ='r', encoding='utf-8')as file:
  csv_file = csv.reader(file)

  first_line = True
  for line in csv_file:
        if first_line:
            first_line = False
            continue

        celeb_heights_entry = CelebHeightsEntry(line)
        celeb_heights_entries_map[celeb_heights_entry.name] = celeb_heights_entry


# check all notion entries
updates: list[UpdateThis] = []

current_notion_entries: list[utils.Entry] = utils.get_notion_entries()
for entry in current_notion_entries:
    # see if notion entry is on celeb heights
    match = process.extractOne(entry.name, celeb_heights_entries_map.keys(), scorer=fuzz.token_sort_ratio, score_cutoff=90)
    
    if match != None:
        celeb_name, percent_match = match
        celeb_heights_entry = celeb_heights_entries_map[celeb_name]

        update_entry: UpdateThis = UpdateThis(entry, celeb_heights_entry, percent_match)

        # see if anything (height or image url) needs to be updated
        if entry.height == None or entry.height != celeb_heights_entry.height:
            update_entry.add_update_needed(OutdatedWrapper(
                outdated = Outdated.HEIGHT,
                current_val = entry.height,
                should_be_val = celeb_heights_entry.height
                ))

        if entry.image_url == None or entry.image_url != celeb_heights_entry.image_url:
            update_entry.add_update_needed(OutdatedWrapper(
                outdated = Outdated.IMAGE,
                current_val = entry.image_url,
                should_be_val = celeb_heights_entry.image_url
                ))

        if update_entry.needs_update():
            updates.append(update_entry)

{'object': 'list', 'results': [{'object': 'page', 'id': '1b622473-cfcf-80ba-8f8c-c17580633239', 'created_time': '2025-03-14T03:55:00.000Z', 'last_edited_time': '2025-03-14T03:55:00.000Z', 'created_by': {'object': 'user', 'id': '9b26488b-3ce7-434b-aca7-88ce0f10f342'}, 'last_edited_by': {'object': 'user', 'id': '9b26488b-3ce7-434b-aca7-88ce0f10f342'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': 'b646c81b-09e4-48b9-8812-681cb5646506'}, 'archived': False, 'in_trash': False, 'properties': {'Image URL': {'id': '%3DAS%3E', 'type': 'url', 'url': 'https://www.celebheights.com/tr/a/adamsavage.jpg'}, 'Tags': {'id': 'AA%3DV', 'type': 'multi_select', 'multi_select': [{'id': '3b2f5ce0-13bb-4429-9bd5-111eb0e5721d', 'name': 'Influencers', 'color': 'default'}, {'id': '7b006f9c-85cc-4787-b56c-e271839c0ad4', 'name': 'Scientists', 'color': 'pink'}, {'id': '64f8052c-ef93-4a57-aecd-4e489f7bde8b', 'name': 'Acting', 'color': 'brown'}]}, 'ID': {'id': 'J%3Clc', 'type': 'unique_

In [3]:
# display updates!
print(f"{len(updates)} updates needed!")

for update in updates:
    print(f"{update.notion_entry.name} --- (found from {update.celeb_heights_entry.name}, {update.percent_match}% match, {update.celeb_heights_entry.url})")

    for outdated in update.updates_needed:
        print(outdated.outdated)
        print(f"currently: \n{outdated.current_val}")
        print(f"should be: \n{outdated.should_be_val}")
        print('---')

    print()

1 updates needed!
Dev --- (found from Dev, 100% match, https://www.celebheights.com/s/-Dev-48461.html)
Outdated.HEIGHT
currently: 
171
should be: 
165
---
Outdated.IMAGE
currently: 
None
should be: 
https://www.celebheights.com/tr/d/dev.jpg
---

